# Test the deployed web application

In this notebook, we test the local web app running inside the Docker container we made previously.

In [ ]:
import numpy as np
import requests
import pandas as pd
import json

from utilities import text_to_json

In [ ]:
docker_login = 'fboylu'
image_name = docker_login + '/mlaksdep'

Run the Docker conatainer in the background and open port 80.

In [ ]:
%%bash --bg -s "$image_name"
docker run -p 80:80 $1

Wait a few seconds for the application to spin up and then check that everything works.

In [ ]:
!curl "http://0.0.0.0:80/"

In [ ]:
!curl "http://0.0.0.0:80/version"

Now, let's use one of the duplicate questions to test our driver.

In [ ]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

In [ ]:
jsontext = text_to_json(text_to_score)
jsontext[:100]

'{"input": "javascript arrays as objects.  possible duplicate: length of javascript object (ie. assoc'

In [ ]:
headers = {'content-type': 'application/json'}
%time r = requests.post('http://0.0.0.0:80/score', data=jsontext, headers=headers)
print(r)
r.json()

Let's try a few more duplicate questions and display their top 3 original matches.

In [9]:
dupes_to_score = dupes_test.iloc[:5,4]

In [10]:
url = 'http://0.0.0.0:80/score'
results = [requests.post(url, data=text_to_json(text), headers=headers) for text in dupes_to_score]

Let's print top 3 matches for each duplicate question.

In [11]:
[results[i].json()['result'][0][0:3] for i in range(0, len(results))]

[[[11922383, 11922384, 0.4618297837830524],
  [750486, 750506, 0.10187547714586409],
  [5223, 6700, 0.0638246129572683]],
 [[14220321, 14220323, 0.8221692467477977],
  [11922383, 11922384, 0.6581886437186476],
  [5223, 6700, 0.6432238154719481]],
 [[14220321, 14220323, 0.9645868077283122],
  [27928, 27943, 0.9222570573208747],
  [13840429, 13840431, 0.7895954858417572]],
 [[27928, 27943, 0.8253809764787108],
  [14220321, 14220323, 0.35839553716891404],
  [23667086, 23667087, 0.2802815688638847]],
 [[203198, 1207393, 0.989876477409501],
  [152975, 153047, 0.5379549302867603],
  [12829963, 12830031, 0.22370925120587182]]]

Next let's quickly check what the request response performance is for the locally running Docker container.

In [12]:
text_data = list(map(text_to_json, dupes_to_score)) # Retrieve the text data

In [13]:
timer_results = list()
for text in text_data:
    res=%timeit -r 1 -o -q requests.post(url, data=text, headers=headers)
    timer_results.append(res.best)

In [14]:
timer_results

[0.03609351180493832,
 0.03917934652417898,
 0.06589280478656293,
 0.03953545484691858,
 0.03342356495559216]

In [15]:
print('Average time taken: {0:4.2f} ms'.format(10**3 * np.mean(timer_results)))

Average time taken: 42.82 ms


In [16]:
%%bash
docker stop $(docker ps -q)

88eb5b85ddd8


We can now [deploy our web application](05_Deploy_On_AKS.ipynb) on AKS.